In [71]:
import pandas as pd
import numpy as np

In [72]:
df = pd.read_csv('CAR DETAILS FROM CAR DEKHO.csv')
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [73]:
df.shape

(4340, 8)

In [74]:
df1 = df.drop('seller_type', axis='columns')
df1.head()

,name,year,selling_price,km_driven,fuel,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Manual,Second Owner


DATA CLEANING

In [75]:
df1.isnull().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
transmission     0
owner            0
dtype: int64

Outlier detection and removal

In [76]:
df1.describe()

,year,selling_price,km_driven
count,4340.000000,4.340000e+03,4340.000000
mean,2013.090783,5.041273e+05,66215.777419
std,4.215344,5.785487e+05,46644.102194
min,1992.000000,2.000000e+04,1.000000
25%,2011.000000,2.087498e+05,35000.000000
50%,2014.000000,3.500000e+05,60000.000000
75%,2016.000000,6.000000e+05,90000.000000
max,2020.000000,8.900000e+06,806599.000000


In [77]:
def remove_car_outliers(df):
    exclude_indices = np.array([])
    
    # Group by each car model
    for name, car_df in df.groupby('name'):
        
        #IQR for 'selling_price'
        q1_price = car_df['selling_price'].quantile(0.25)
        q3_price = car_df['selling_price'].quantile(0.75)
        iqr_price = q3_price - q1_price
        lower_bound_price = q1_price - 1.5 * iqr_price
        upper_bound_price = q3_price + 1.5 * iqr_price
        
        price_outliers = car_df[(car_df['selling_price'] < lower_bound_price) | (car_df['selling_price'] > upper_bound_price)].index
        exclude_indices = np.append(exclude_indices, price_outliers)
        
        #IQR for 'km_driven'
        q1_km = car_df['km_driven'].quantile(0.25)
        q3_km = car_df['km_driven'].quantile(0.75)
        iqr_km = q3_km - q1_km
        lower_bound_km = q1_km - 1.5 * iqr_km
        upper_bound_km = q3_km + 1.5 * iqr_km
        
        # Find indices of km outliers for this group
        km_outliers = car_df[(car_df['km_driven'] < lower_bound_km) | (car_df['km_driven'] > upper_bound_km)].index
        exclude_indices = np.append(exclude_indices, km_outliers)

    # Get a unique list of indices to drop
    unique_exclude_indices = np.unique(exclude_indices)
    
    cleaned_df = df.drop(unique_exclude_indices, axis='index')
    return cleaned_df

In [78]:
df2 = remove_car_outliers(df1)
df2.shape

(4081, 7)

In [79]:
df2.describe()

,year,selling_price,km_driven
count,4081.000000,4.081000e+03,4081.000000
mean,2013.115168,5.084103e+05,64725.769419
std,4.224400,5.892203e+05,43439.287683
min,1992.000000,2.000000e+04,101.000000
25%,2011.000000,2.100000e+05,35000.000000
50%,2014.000000,3.600000e+05,60000.000000
75%,2017.000000,6.000000e+05,90000.000000
max,2020.000000,8.900000e+06,560000.000000


In [80]:
df2.head()

,name,year,selling_price,km_driven,fuel,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Manual,Second Owner


Handling Categorical Values

In [81]:
df2.name.nunique()

1491

In [82]:
df2.fuel.unique()

array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object)

In [83]:
df2.transmission.unique()

array(['Manual', 'Automatic'], dtype=object)

In [84]:
df2.owner.unique()

array(['First Owner', 'Second Owner', 'Fourth & Above Owner',
       'Third Owner', 'Test Drive Car'], dtype=object)

In [85]:
df2.name = df2.name.apply(lambda x: x.split())
name_stats = df2['name'].value_counts(ascending=False)
name_stats

name
[Maruti, Swift, Dzire, VDI]              67
[Maruti, Alto, 800, LXI]                 54
[Maruti, Alto, LXi]                      46
[Maruti, Alto, LX]                       35
[Hyundai, EON, Era, Plus]                35
                                         ..
[Honda, Amaze, S, Petrol, BSIV]           1
[Tata, Indigo, CS, LS, (TDI), BS-III]     1
[Mahindra, KUV, 100, G80, K4, Plus]       1
[Tata, Indica, V2, DLS, BSII]             1
[Mahindra, XUV500, W10, AWD]              1
Name: count, Length: 1491, dtype: int64

In [86]:
df3 = df2.copy()

In [87]:
df3['brand'] = df3['name'].apply(lambda x: x[0])
df3.drop('name', axis='columns', inplace=True)
df3.head()

,year,selling_price,km_driven,fuel,transmission,owner,brand
0,2007,60000,70000,Petrol,Manual,First Owner,Maruti
1,2007,135000,50000,Petrol,Manual,First Owner,Maruti
2,2012,600000,100000,Diesel,Manual,First Owner,Hyundai
3,2017,250000,46000,Petrol,Manual,First Owner,Datsun
4,2014,450000,141000,Diesel,Manual,Second Owner,Honda


In [88]:
df3.brand.nunique()

29

1) Applying One hot encoding to brand , fuel , transmission

2) Applying label encoding to owner

In [89]:
dummies = pd.get_dummies(df3[['brand', 'fuel', 'transmission']], drop_first=True)
df4 = pd.concat([df3, dummies], axis='columns')
df4.head()

,year,selling_price,km_driven,fuel,transmission,owner,brand,brand_Audi,brand_BMW,brand_Chevrolet,...,brand_Skoda,brand_Tata,brand_Toyota,brand_Volkswagen,brand_Volvo,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,transmission_Manual
0,2007,60000,70000,Petrol,Manual,First Owner,Maruti,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,2007,135000,50000,Petrol,Manual,First Owner,Maruti,False,False,False,...,False,False,False,False,False,False,False,False,True,True
2,2012,600000,100000,Diesel,Manual,First Owner,Hyundai,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,2017,250000,46000,Petrol,Manual,First Owner,Datsun,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,2014,450000,141000,Diesel,Manual,Second Owner,Honda,False,False,False,...,False,False,False,False,False,True,False,False,False,True


In [90]:
owner_map = {
    'Test Drive Car': 0,
    'First Owner': 1,
    'Second Owner': 2,
    'Third Owner': 3,
    'Fourth & Above Owner': 4
}

df4['owner'] = df4['owner'].map(owner_map)

In [91]:
df4.head()

,year,selling_price,km_driven,fuel,transmission,owner,brand,brand_Audi,brand_BMW,brand_Chevrolet,...,brand_Skoda,brand_Tata,brand_Toyota,brand_Volkswagen,brand_Volvo,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,transmission_Manual
0,2007,60000,70000,Petrol,Manual,1,Maruti,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,2007,135000,50000,Petrol,Manual,1,Maruti,False,False,False,...,False,False,False,False,False,False,False,False,True,True
2,2012,600000,100000,Diesel,Manual,1,Hyundai,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,2017,250000,46000,Petrol,Manual,1,Datsun,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,2014,450000,141000,Diesel,Manual,2,Honda,False,False,False,...,False,False,False,False,False,True,False,False,False,True


In [92]:
df5 = df4.copy()

In [93]:
df5.drop(['brand', 'fuel', 'transmission'], axis='columns', inplace=True)
df5.head()

,year,selling_price,km_driven,owner,brand_Audi,brand_BMW,brand_Chevrolet,brand_Daewoo,brand_Datsun,brand_Fiat,...,brand_Skoda,brand_Tata,brand_Toyota,brand_Volkswagen,brand_Volvo,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,transmission_Manual
0,2007,60000,70000,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,2007,135000,50000,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
2,2012,600000,100000,1,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,2017,250000,46000,1,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,True
4,2014,450000,141000,2,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True


BUILDING THE MODEL

In [94]:
X = df5.drop('selling_price', axis='columns')
y = df5.selling_price

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [97]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [98]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['squared_error','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(),
            'params': {
                'n_estimators': [50, 100, 150]
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X_train_scaled,y_train)

,model,best_score,best_params
0,linear_regression,0.660471,{'fit_intercept': True}
1,decision_tree,0.762892,"{'criterion': 'squared_error', 'splitter': 'ra..."
2,random_forest,0.793830,{'n_estimators': 100}


So Random Forest Regressor is the best performing model

In [99]:
from sklearn.preprocessing import StandardScaler

# Define the numerical columns to be scaled
numerical_features = ['year', 'km_driven', 'owner']

# Create and fit the scaler on the training data
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

# Use the same scaler to transform the test data
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# Now, train the model on the complete and correctly scaled training data
model = RandomForestRegressor(n_estimators=50)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50)

Exporting the model to be further used by the flask server

In [100]:
import pickle
with open('car_price_prediction.pickle', 'wb') as f:
    pickle.dump(model, f)

Exporting columns of the model trained

In [101]:
import json
columns = {
    'data_columns' : [col for col in X.columns]  # <-- .lower() is removed
}
with open("columns.json", "w") as f:
    f.write(json.dumps(columns))

In [102]:
import pickle
with open('scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)